### DATA SCRAPING OF TRANSFERMARKET AND FBREF FOR SPORTS ANALYTICS


Import libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import time

Obtain Transfermarkt and FBref Websites for Scraping (Manually Selected Websites) - Automate the Process in the Future

In [3]:
excel_file = 'transfer_links.xlsx'

urls = pd.read_excel(excel_file, header=None, names=['Transfermarkt', 'Fbref', 'League'])
urls_transfermarkt = urls['Transfermarkt'].to_numpy()
urls_fbref = urls['Fbref'].to_numpy()
leagues = urls['League'].to_numpy()

Define Function to Get HTML Code from a Website

In [4]:
def get_Soup(page):
    headers = {'User-Agent':
                 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
    return pageSoup

Define Function to Get Table HTML Code from Transfermarkt Website

In [5]:
def get_information (pageSoup):

    rows_bi = pageSoup.find_all("td", {"class": "posrela"})
    rows_ai = pageSoup.find_all("td", {"class": "zentriert"})
    rows_mi = pageSoup.find_all("td", {"class": "rechts hauptlink"})

    grouped_rows = []
    for i in range(0, len(rows_ai), 8):
        group = rows_ai[i:i+8]
        grouped_rows.append(group)

    return rows_bi, grouped_rows, rows_mi

Define Function to Get Transfermarkt Player Information

In [6]:
def get_players_information (league, team, rows_bi, grouped_rows, rows_mi):

    players = []

    for row in rows_bi:
        dict = {}

        dict["Liga"] = league

        dict['Equipo'] = team

        # Extract player name
        dict["Nombre"] = row.find_all('td')[1].text.strip()

        # Extract image
        dict["Imagen"] = row.find('img', {'data-src': True})['data-src']

        # Extract position
        dict["Posicion"] = row.find_all('td')[2].text.strip()


        try:
            dict['Lesionado'] = row.find_all('td')[1].find('span').get('title')
        except:
            dict['Lesionado'] = np.nan

        try:
            dict['Estado jugador'] = row.find('a', class_='hide-for-small').get('title')
        except:
            dict['Estado jugador'] = np.nan

        players.append(dict)

    index = 0
    for row in grouped_rows:

        dict = players[index]

        # Extract player number
        try:
            dict["Numero"] = row[0].text
        except:
            dict["Numero"] = np.nan

        # Extract dob
        try:
            dict["Fecha de Nacimiento"] = row[1].text[:10]
        except:
            dict["Fecha de Nacimiento"] = np.nan

        # Extract age
        try:
            dict["Edad"] = row[1].text[12:14]
        except:
            dict["Edad"] = np.nan


        # Extract nationality (JUST THE FIRST ONE)
        try:
            dict["Nacionalidad"] = row[2].img['alt']
        except:
            dict["Nacionalidad"] = np.nan

        # Extract altura
        try:
            dict["Altura"] = row[3].text
        except:
            dict["Altura"] = np.nan

        # Extract pie
        try:
            dict["Pie"] = row[4].text
        except:
            dict["Pie"] = np.nan

        # Extract fichado
        try:
            dict["Fecha de incorporacion"] = row[5].text
        except:
            dict["Fecha de incorporacion"] = np.nan

        # Extract anterior equipo
        try:
            dict["Anterior Equipo"] = row[6].img['alt']
        except:
            dict["Anterior Equipo"] = np.nan

        # Extract contrato
        try:
            dict["Fecha de fin de contrato"] = row[7].text
        except:
            dict["Fecha de fin de contrato"] = np.nan

        index += 1

    index = 0
    for row in rows_mi:
        dict = players[index]

        # Extract valor de mercado
        dict["Valor de mercado"] = row.text

        index += 1

    return players

Get Player Information from All Transfermarkt Websites (7 min)

In [7]:
all_players = np.array([])
index = 0

for page in urls_transfermarkt:
    try:
        pageSoup = get_Soup(page)
        league = leagues[index]
        team = urls_fbref[index].split('/')[-1].replace('-', ' ').replace(' Stats', '')
        rows_bi, grouped_rows, rows_mi = get_information(pageSoup)
        players = get_players_information(league, team, rows_bi, grouped_rows, rows_mi)
        all_players = np.concatenate((all_players, players))
    except Exception as e:
        print(page)
        print(e)
    index += 1

Define Functions to transform DataFrame columns

In [8]:
def n_int(n_str):
    try:
        return int(n_str)
    except:
        return np.nan

def altura_to_m(string):
    try:
        return int(string[0] + string[2:4])/100
    except:
        return np.nan

def value_to_int(value):
    try:
        cleaned_value = value.replace(',', '').replace(' mill. €', '')
        converted_value = int(cleaned_value)/100
        return converted_value
    except:
        try:
            cleaned_value = value.replace(',', '').replace(' mil €', '')
            converted_value = int(cleaned_value)/1000
            return converted_value
        except:
            return None

def convert_to_date(date_string):
    try:
        date_format = "%d/%m/%Y"
        converted_date = datetime.strptime(date_string.strip(), date_format).date()
        return converted_date
    except:
        return None

Transfermarkt DataFrame (Clean certain Transfermarkt Dataframe columns)

In [9]:
df_transfermarkt = pd.DataFrame.from_records(all_players)

df_transfermarkt['Altura'] = df_transfermarkt["Altura"].apply(altura_to_m)
df_transfermarkt["Valor de mercado"] = df_transfermarkt["Valor de mercado"].apply(value_to_int)
df_transfermarkt["Fecha de Nacimiento"] = df_transfermarkt["Fecha de Nacimiento"].apply(convert_to_date)
df_transfermarkt["Fecha de incorporacion"] = df_transfermarkt["Fecha de incorporacion"].apply(convert_to_date)
df_transfermarkt["Fecha de fin de contrato"] = df_transfermarkt["Fecha de fin de contrato"].apply(convert_to_date)

Define Function to Get Player Information from FBref Website

In [10]:
def get_players_statistics(page, pageSoup):
    data_dict = {}

    sections = {
        'gk_info': 'all_stats_keeper_adv',
        'shooting_info': 'all_stats_shooting',
        'passing_info': 'all_stats_passing',
        'pass_type_info': 'all_stats_passing_types',
        'gca_info': 'all_stats_gca',
        'defense_info': 'all_stats_defense',
        'possession_info': 'all_stats_possession',
        'playing_time_info': 'all_stats_playing_time',
        'misc_info': 'all_stats_misc'
    }


    dict_all = {}
    club_name = page.split('/')[-1].replace('-', ' ').replace(' Stats', '')
    for key, value in sections.items():
        try:
            section_info = pageSoup.find('div', id=value)
            section_col = np.unique([td['data-stat'] for td in section_info.find_all('td')])
            table = section_info.find('tbody')
            rows = table.find_all('tr')
            for row in rows:
                data_dict = {}
                try:
                    player = row.find('th', attrs={'data-stat': 'player'}).text
                    data_dict['player'] = player if player else np.nan
                    data_dict['team'] = club_name
                except:
                    data_dict['player'] = np.nan
                    data_dict['team'] = np.nan

                for stat in section_col:
                    try:
                        value = row.find('td', attrs={'data-stat': stat}).text
                        data_dict[stat] = value if value else np.nan
                    except:
                        data_dict[stat] = np.nan

                if data_dict['player'] in list(dict_all.keys()):
                    dict_all[data_dict['player']].update(data_dict)
                else:
                    dict_all[data_dict['player']] = data_dict
        except:
            print((page, value))

    return dict_all

Fbref DataFrame (3 hours)

In [ ]:
overall_df = pd.DataFrame()

for page in urls_fbref:
    try:
        pageSoup = get_Soup(page)
        dict_all = get_players_statistics(page, pageSoup)
        player_list = []
        for player, info in dict_all.items():
            player_info = {'Player': player}
            player_info.update(info)
            player_list.append(player_info)
        df_fbref = pd.DataFrame(player_list)
        overall_df = pd.concat([overall_df, df_fbref], ignore_index=True)  # Concatenate the current dataframe with the overall dataframe
        overall_df.to_csv('fbref_stats.csv')

        time.sleep(1)
    except:
        print(page)


Read Saved Fbref DataFrame

In [12]:
df_fbref = pd.read_csv('fbref_stats.csv', index_col=0)

Merge Transfermarkt & Fbref DataFrames by player name and team

In [13]:
stats_df = pd.merge(df_transfermarkt, df_fbref, left_on=['Nombre', 'Equipo'], right_on=['player', 'team'], how='inner')

Save merged DataFrame

In [14]:
stats_df.to_csv('transfer.csv', index = False)